In [4]:
!python --version

Python 3.11.4


In [2]:
!pip freeze > requirements.txt

In [ ]:
#!pip install googletrans==3.1.0a0
#!pip install langdetect
!pip install ai4bharat-transliteration

In [1]:
#from googletrans import Translator
#from langdetect import detect
from ai4bharat.transliteration import XlitEngine
import os
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')
print('Required Libraries Imported')

Required Libraries Imported


In [2]:
text = 'पूनम Devi'
#translator = Translator()
#translation = translator.detect(text)
#print(translation.lang)
xlited=[]
e = XlitEngine(src_script_type="indic")
out = e.translit_sentence(text, lang_code='hi')
print(out)

Initializing Multilingual model for transliteration


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL argparse.Namespace was not an allowed global by default. Please use `torch.serialization.add_safe_globals([argparse.Namespace])` or the `torch.serialization.safe_globals([argparse.Namespace])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by defa

In [ ]:
data1 = pd.read_excel('Nassscom/1.0/NF 1.0 Local Language Names.xlsx')
data2 = pd.read_excel('Nassscom/1.0/Test Translit.xlsx')
data2

In [ ]:
def translit(col):
  translator = Translator()
  e = XlitEngine(src_script_type="indic", beam_width=10, rescore=False)

  transliterated = []

  for s in col:
    if len(s.split(' '))>1:
      ts = []
      for w in s.split(' '):
        translation = translator.detect(w)
        try:
          ts.append(e.translit_word(w, lang_code = translation.lang, topk=1))
        except NotImplementedError:
          ts.append(w)

      transliterated.append(' '.join(ts))
    else:
      translation = translator.detect(s)
      try:
        transliterated.append(e.translit_sentence(s, lang_code = translation.lang))
      except NotImplementedError:
        transliterated.append(s)


  return transliterated



In [ ]:
translator = Translator()
e = XlitEngine(src_script_type="indic", beam_width = 5, rescore=False)

def translit(x):
  lang = translator.translate(x)
  lang_code = lang.src

  try:
    translited = e.translit_sentence(x, lang_code = lang_code)
    print("Transliterated : ",translited)
    return translited
        
  except NotImplementedError:
    ts = []
    for w in x.split(' '):
      lang = translator.translate(w)
      lang_code = lang.src
      try:
        ts.append(e.translit_word(w, lang_code = lang_code, topk=1)[0])
      except NotImplementedError:
        ts.append(w)
    s = ' '.join(ts)
    print("Not Transliterated : ", s)
    return s

In [ ]:
data1 = pd.read_csv(r"C:\Python\read\cases_report_Nasscom1.0_All_(All States)_2024-05-17.csv")
data1['Transliterated'] = data1['Citizen Name'].apply(lambda x: translit(x))
data1.head()

In [ ]:
data1.to_excel('Nassscom/1.0/NF 1.0 Transliterated Local Language Names.xlsx', index=False)
print("Export to excel success!")

In [25]:
t = Translator()
txlated = t.translate("उदम सिंह नगर")
txlated.text

'Udam Singh Nagar'

# Using Sarvam.ai for Transliteration

In [4]:
import pandas as pd, numpy as np
df = pd.read_excel(r'F:\Haqdarshak Data\DBS\Phase 2\DBS 2.0 - All Data - 19-09-2025.xlsx', sheet_name='Schemes Data')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317176 entries, 0 to 317175
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Case Id                         317175 non-null  object        
 1   Createdon                       317176 non-null  datetime64[ns]
 2   Scheme/Doc                      317176 non-null  object        
 3   Scheme/Doc GUID                 317176 non-null  object        
 4   Status                          317176 non-null  object        
 5   Docket Submitted Date           291157 non-null  datetime64[ns]
 6   Benefit received Date           270462 non-null  datetime64[ns]
 7   HD Suspected Cases              317176 non-null  object        
 8   State                           317176 non-null  object        
 9   District                        317176 non-null  object        
 10  Citizen GUID                    317176 non-null  object 

In [5]:
dbs2_OR = df[df.State == 'Odisha']
dbs2_OR.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48427 entries, 0 to 317065
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Case Id                         48427 non-null  object        
 1   Createdon                       48427 non-null  datetime64[ns]
 2   Scheme/Doc                      48427 non-null  object        
 3   Scheme/Doc GUID                 48427 non-null  object        
 4   Status                          48427 non-null  object        
 5   Docket Submitted Date           43266 non-null  datetime64[ns]
 6   Benefit received Date           35652 non-null  datetime64[ns]
 7   HD Suspected Cases              48427 non-null  object        
 8   State                           48427 non-null  object        
 9   District                        48427 non-null  object        
 10  Citizen GUID                    48427 non-null  object        
 11  Citize

In [18]:
from sarvamai import SarvamAI
import os, string

key = open(r'C:\Python\sarvamAPIKey.txt').read()
os.environ['SARVAM_API_KEY']=key

sai = SarvamAI(api_subscription_key=key,)

text = 'पूनम Devi'

def sai_transliterate(text, src='auto', dest="en-IN"):
    responce = sai.text.transliterate(
        input=text,
        source_language_code=src,
        target_language_code=dest,
        spoken_form=True
    )

    return responce.transliterated_text

small_alpha = list(string.ascii_lowercase)

In [21]:
dbs2_OR['xlited_hi'] = dbs2_OR['Citizen Name'].apply(lambda x: sai_transliterate(x,'en-IN',"od-IN") if x[0].lower() in small_alpha else x )
dbs2_OR.head()

ApiError: headers: {'server': 'Microsoft-Azure-Application-Gateway/v2', 'date': 'Sun, 21 Sep 2025 20:38:46 GMT', 'content-type': 'text/html', 'content-length': '183', 'connection': 'keep-alive', 'access-control-allow-origin': '*', 'strict-transport-security': 'max-age=31536000; includeSubDomains; preload'}, status_code: 502, body: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>Microsoft-Azure-Application-Gateway/v2</center>
</body>
</html>


In [ ]:
dbs2_OR.to_excel(r'C:\Python\export\DBS2_xlited.xlsx', sheet_name='Schemes Data', index=False )